In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
corner_track_params = dict(maxCorners=10,minDistance=5,qualityLevel=0.4,blockSize=7)
corner_track_params

{'maxCorners': 10, 'minDistance': 5, 'qualityLevel': 0.4, 'blockSize': 7}

In [3]:
lk_params = dict(winSize=(200,200),maxLevel=2,criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,10,0.03))
lk_params

{'winSize': (200, 200), 'maxLevel': 2, 'criteria': (3, 10, 0.03)}

In [4]:
# OPTICAL FLOW VIDEO CAPTURE
cap = cv2.VideoCapture(0)
ret, prev_frame = cap.read()
prev_gray = cv2.cvtColor(prev_frame,cv2.COLOR_BGR2GRAY)
prev_pts = cv2.goodFeaturesToTrack(prev_gray,mask=None,**corner_track_params)
mask = np.zeros_like(prev_frame)

while True:
    ret, curr_frame = cap.read()
    curr_gray = cv2.cvtColor(curr_frame,cv2.COLOR_BGR2GRAY)
    next_pts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray,curr_gray,prev_pts,None,**lk_params)
    good_new = next_pts[status==1]
    good_prev = prev_pts[status==1]
    
    for i,(prev,new) in enumerate(zip(good_prev,good_new)):
        x_prev, y_prev = prev.ravel()
        x_new, y_new = new.ravel()
        x_new, y_new = int(x_new), int(y_new)
        x_prev, y_prev = int(x_prev), int(y_prev)
        mask = cv2.line(mask,(x_new,y_new),(x_prev,y_prev),(0,255,0),5)
        curr_frame = cv2.circle(curr_frame,(x_new,y_new),8,(255,0,0),-1)
    
    img = cv2.add(curr_frame,mask)
    cv2.imshow('Image Tracking',img)
    
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break
    
    prev_gray = curr_gray.copy()
    prev_pts = good_new.reshape(-1,1,2)

cap.release()
cv2.destroyAllWindows()

In [5]:
# OPTICAL DENSE FLOW VIDEO CAPTURE
cap = cv2.VideoCapture(0)

ret, prev_frame = cap.read()

prev_gray = cv2.cvtColor(prev_frame,cv2.COLOR_BGR2GRAY)

hsv_mask = np.zeros_like(prev_frame)
hsv_mask[:,:,1] = 255

while True:
    ret, curr_frame = cap.read()
    curr_gray = cv2.cvtColor(curr_frame,cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(prev_gray,curr_gray,None,0.5,3,15,3,5,1.2,0)
    mag, ang = cv2.cartToPolar(flow[:,:,0],flow[:,:,1],angleInDegrees=True)
    hsv_mask[:,:,0] = ang / 2
    hsv_mask[:,:,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    hsv_bg = cv2.cvtColor(hsv_mask,cv2.COLOR_HSV2RGB)
    cv2.imshow('Optical Dense Flow Image Tracking',hsv_bg)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break  
        
    prev_gray = curr_gray

cap.release()
cv2.destroyAllWindows()